# Nengo structure draft

So upon further investigation the nengo documentation is actually... not that nice. Trying my best though.

https://www.nengo.ai/nengo/v3.2.0/frontend-api.html?highlight=nengo%20convolution#nengo.Convolution

In [1]:
import nengo
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Parameters from the paper
n_neurons = 40
f_maps = 50
window_size = (6, 40)

mean = 0.8
std = 0.05

threshold = 23
thresh_config = nengo.presets.ThresholdingEnsembles(threshold) # Set the threshold

In [4]:
model = nengo.Network()

with model:

    # Not entirely sure about these dimensions yet
    input_layer = nengo.Node(input, size_out=input.Shape[0] * input.Shape[1])
    pre = nengo.Ensemble(n_neurons, dimensions = input.Shape[0] * input.Shape[1])
    post = nengo.Ensemble(n_neurons, dimensions = input.Shape[0] * input.Shape[1]) # ??
    
    # This looks fairly solid to me though
    transform = nengo.Convolution(
                n_filters = f_maps,
                input_shape = (input.Shape[0], input.Shape[1]),
                kernel_size = window_size,
                strides = (1,1), # I think it's 2D convolution based on the other parameters at least
                padding="same",
                channels_last = True,
                init = nengo.dists.Gaussian(mean, std)
            )
    
    # And then you can apply it on the input as preprocessing step
    conv_conn = nengo.Connection(input_layer, pre, transform = transform)
    
    # Learning rule
    learn_conn = nengo.Connection(
        pre, post,
        learning_rule_type = nengo.BCM(learning_rate=5e-10), # Change this later
        solver = nengo.solvers.LstsqL2(weights=True)
    )
    
    
    #Probes and such
    input_probe = nengo.Probe(input_layer)
    pre_probe = nengo.Probe(pre, synapse=0.01)
    post_probe = nengo.Probe(post, synapse=0.01)

From the BCM docs: This seems consistent with the idea that it potentiates active synapses, and depresses non-active synapses.
This seems similar at least to STDP? We can use it to check how well it works.

http://compneuro.uwaterloo.ca/publications/bekolay2011.html

http://compneuro.uwaterloo.ca/publications/bekolay2013.html

What I expect the final results should sort of look like (this example is with MNIST):

![MNIST results](https://www.nengo.ai/nengo-examples/_images/loihi_mnist-convnet_18_1.png)